In [ ]:
import pandas as pd
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from nltk.tokenize import word_tokenize
frase = 'Esse produto é muito ruim! @vendedor não retornou as minhas reclamações, desapontado :('
word_tokenize(frase)

['Esse',
 'produto',
 'é',
 'muito',
 'ruim',
 '!',
 '@',
 'vendedor',
 'não',
 'retornou',
 'as',
 'minhas',
 'reclamações',
 ',',
 'desapontado',
 ':',
 '(']

In [ ]:
from nltk.tokenize import TweetTokenizer
text_tokenizer = TweetTokenizer()
frase = 'Esse produto é muito ruim! @vendedor não retornou as minhas reclamações, desapontado :('
text_tokenizer.tokenize(frase)

['Esse',
 'produto',
 'é',
 'muito',
 'ruim',
 '!',
 '@vendedor',
 'não',
 'retornou',
 'as',
 'minhas',
 'reclamações',
 ',',
 'desapontado',
 ':(']

In [ ]:
def to_lower_case(texto):
  return texto.lower()

def limpar_texto(texto):
    texto = re.sub(r"http\S+", "", texto).replace('.','')
    texto = re.sub(r'^.+@[^\.].*\.[a-z]{2,}$', 'emailaddress', texto)
    texto = re.sub(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$','webaddress', texto)
    texto = re.sub(r'£|\$', 'moneysymb', texto)
    texto = re.sub(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$', 'phonenumbr', texto)
    texto = re.sub(r'\d+(\.\d+)?', 'numbr', texto)
    texto = re.sub(r'\s+', ' ', texto)
    texto = re.sub(r'^\s+|\s+?$', '', texto)
    return (texto)

from nltk.corpus import stopwords
from string import punctuation
stopwords_custom = set(stopwords.words('portuguese') + list(punctuation))
stopwords_custom.remove('não')
def remover_stopwords(texto):
  palavras = [i for i in texto.split() if not i in stopwords_custom]
  return (" ".join(palavras))

def pre_processar_dados(texto):
  texto = to_lower_case(texto)
  texto = limpar_texto(texto)
  texto = remover_stopwords(texto)
  return texto

In [ ]:
data_treinamento = pd.read_csv('drive/MyDrive/Faculdade/TCC/Algoritmo/bases_de_dados/base_dados_treinamento.csv',
                              sep=';',
                              encoding='utf-8',
                              on_bad_lines='skip')
data_treinamento.head()

,texto,sentimento
0,@Tixaa23 14 para eu ir :),Positivo
1,@drexalvarez O meu like eu já dei na época :),Positivo
2,Eu só queria conseguir comer alguma coisa pra ...,Positivo
3,:D que lindo dia !,Positivo
4,"@Primo_Resmungao Pq da pr jeito!!é uma ""oferta...",Positivo


In [ ]:
data_treinamento['sentimento'].value_counts()

Negativo    9824
Positivo    5883
Name: sentimento, dtype: int64

In [ ]:
textos_treinamento = []
for texto in data_treinamento['texto']:
  print('Antes: ' + texto)
  texto_pre_processado = pre_processar_dados(texto)
  print('Depois: '+ texto_pre_processado)
  textos_treinamento.append(texto_pre_processado)

classificacao_treinamento = data_treinamento['sentimento']

A saída de streaming foi truncada nas últimas 5000 linhas.
Antes: Quem te pegaria curte aqui — 0 :( https://t.co/9nscfJk5uI
Depois: pegaria curte aqui — numbr :(
Antes: @lucaskaaue eu quero um coalaaa e um bicho preguiça :(((
Depois: @lucaskaaue quero coalaaa bicho preguiça :(((
Antes: @Jimultisa quero att :(( amei a au
Depois: @jimultisa quero att :(( amei au
Antes: Meninas que são abandonadas pela namorada no dia do aniversário de namoro :(
Depois: meninas abandonadas namorada dia aniversário namoro :(
Antes: @simaira_ Que aconteceu? :(
Depois: @simaira_ aconteceu? :(
Antes: Eu to muito sentimental hoje :(
Depois: to sentimental hoje :(
Antes: @Gabisalvis @PedrinZika1001 mals pedrinho :(
Depois: @gabisalvis @pedrinzikanumbr mals pedrinho :(
Antes: @anthousai_ Junte. Eu te ajudo. :( GRRR. ????
Depois: @anthousai_ junte ajudo :( grrr ????
Antes: @mikokonat Awww TNC :(
Depois: @mikokonat awww tnc :(
Antes: @luizagmendes tô com sdd de beber contigo :(
Depois: @luizagmendes tô sdd beber c

In [ ]:
base_dados_aplicacao = pd.read_csv('drive/MyDrive/Faculdade/TCC/Algoritmo/bases_de_dados/base_dados_analise.csv',
                                  sep=';',
                                  encoding='utf-8',
                                  on_bad_lines='skip')
base_dados_aplicacao.head()

,Feedback
0,"Eu sempre quis esse livro, eu pedi também os o..."
1,"Amo essa série, é a minha preferida, a cada li..."
2,Leitura rápida
3,Os livros de Morgan Rhodes são cheios de avent...
4,produto chegou em ótima qualidade. muito vale ...


In [ ]:
textos_aplicacao = []
for texto in base_dados_aplicacao['Feedback']:
  print('Antes: ' + texto)
  texto_pre_processado = pre_processar_dados(texto)
  print('Depois: '+ texto_pre_processado)
  textos_aplicacao.append(texto_pre_processado)

In [ ]:
vetorizador = CountVectorizer(analyzer='word', tokenizer=text_tokenizer.tokenize)

In [ ]:
frequencia_treinamento = vetorizador.fit_transform(textos_treinamento)

In [ ]:
frequencia_treinamento

In [ ]:
modelo = MultinomialNB()
modelo.fit(frequencia_treinamento, classificacao_treinamento)

In [ ]:
frequencia_aplicacao = vetorizador.transform(textos_aplicacao)

In [ ]:
modelo_treinado = modelo.predict(frequencia_aplicacao)
textos_analisados = zip(textos_aplicacao, modelo_treinado)

In [ ]:
resultados = []
for texto,classificao_analisada in textos_analisados:
  resultado = {
      "Texto analisado": texto,
      "Classificação": classificao_analisada
  }
  resultados.append(resultado)

In [ ]:
dataframe_resultado = pd.DataFrame(resultados)
dataframe_resultado